In [ ]:
Stacked_Autoencoders_MNIST

In [ ]:
# Input Data 
import tensorflow as tf
import sys
from tensorflow.examples.tutorials.mnist import input_data
from functools import partial
import matplotlib.pyplot as plt

# Load the data
mnist = input_data.read_data_sets("MNIST_data")

In [ ]:
# Stacked Autoencoder Network
# Reset the graph and create saver
tf.reset_default_graph()

# Initialize the parameters
n_inputs = 28*28
n_hidden1 = 600
n_hidden2 = 300
n_hidden3 = n_hidden1
n_outputs = n_inputs
learning_rate = 0.01
l2_reg = 0.0001

# Placeholder to keep the input data
X = tf.placeholder(tf.float32, shape = [None, n_inputs])

# Adding the Layers
he_init = tf.contrib.layers.variance_scaling_initializer()
l2_regularizer = tf.contrib.layers.l2_regularizer(l2_reg)
my_dense_layer = partial(tf.layers.dense, activation = tf.nn.relu, kernel_initializer = he_init,
                        kernel_regularizer = l2_regularizer)
hidden1 = my_dense_layer(X, n_hidden1)
hidden2 = my_dense_layer(hidden1, n_hidden2)
hidden3 = my_dense_layer(hidden2, n_hidden3)
outputs = my_dense_layer(hidden3, n_outputs, activation = None)

# Calculating Loss and doing optimization
reconstruction_loss = tf.reduce_mean(tf.square(outputs - X))
reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
loss = tf.add_n([reconstruction_loss] + reg_losses)
optimizer = tf.train.AdamOptimizer(learning_rate)
training_op = optimizer.minimize(loss)

# Executing the Computational Graph
init = tf.global_variables_initializer()

saver = tf.train.Saver()

n_epochs = 7
batch_size = 150
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        n_batches = mnist.train.num_examples // batch_size
        for iteration in range(n_batches):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict = {X: X_batch})
            loss_train = reconstruction_loss.eval(feed_dict = {X: X_batch})
        print(epoch, "\tMSE: ", loss_train)
    saver.save(sess, "./my_model_all_layers.ckpt")


In [ ]:
# Plotting the Input and Ouput Data
n_digits = 2

# To plot figures
% matplotlib inline
import matplotlib.pyplot as plt

plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

def plot_image(image, shape = [28, 28]):
    plt.imshow(image.reshape(shape), cmap = "Greys", interpolation = "nearest")
    plt.axis("off")
    
with tf.Session() as sess:
    saver.restore(sess, "./my_model_all_layers.ckpt")
    X_test = mnist.test.images[:n_digits]
    outputs_val = outputs.eval(feed_dict = {X :X_test})
    fig = plt.figure(figsize = (8,6))
    for index in range(n_digits):
        plt.subplot(n_digits, 2, index *2 +1)
        plot_image(X_test[index])
        plt.subplot(n_digits, 2, index*2+2)
        plot_image(outputs_val[index])